# Apresentação ✒️

Notebook destinado ao estudo do framework LangChain. Nesse notebook, trago a explicação e implementação dos tipos de memórias usuais do LangChain, que visam prover ao modelo de LLM de uso a capacidade de se lembrar das informações geradas com o usuário, de modo a oferecer a ele melhor experiência.

Nesse sentido, esse notebook conta com a BufferyMemory, WindowMemory, TokensBufferyMemory e SummaryMemory. Além dessas, é oportuno mencionar a Vector Data Memory e a Entity Memory.

A primeira transforma as informações geradas em vetores, que podem ser armazenados num espaço vetorial, permitindo a escalabilidade da implementação e eficiência, enquanto que a segunda é mais atrelada a conseguir se lembrar das entidades que permeiam a informação e os elementos a eles relacionados.

Por serem um pouco mais complexos de serem implementados e nesse momento não estarem no contexto do curso da DeepLearningAI, não estarão nesse notebook.

## Biblioteca 📚

In [1]:
!pip install google-generativeai -q

!pip install langchain -q

!pip install langchain_google_genai -q

In [39]:
import warnings
warnings.filterwarnings('ignore')

import google.generativeai as genai

import os
import langchain

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import ChatPromptTemplate

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryBufferMemory


In [3]:
os.environ['GOOGLE_API_KEY'] = 'sua-api-key'

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

## Instanciando a LLM de uso 🤖

In [21]:
# Criando o modelo de llm utilizado com o google gemini.

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-pro-latest",
    temperature = 0.8,
    max_output_tokens=2000
)

## BuferryMemory

Esse tipo de memória retem toda a informação gerada pela IA e o usuário, na forma que sempre que inicia-se uma nova conversa, aquilo que fora gerado é informado, como forma de prover à LLM um senso de memória, dado que por padrão elas não apresentam essa funcionalidade.

Desse modo, para cada interação que o usuário realiza com a IA, essa vai sendo armazenada na memória, a qual é enviada para a LLM. Um ponto positivo acerca dessa abordagem é que a LLM tende a 'lembrar' de tudo o que fora interagido, mas um dos pontos negativos desse uso é que aumenta substancialmente os custos de processamento da LLM, que pode reverter em custos monetários, dado a cobrança por Token gerado.

In [5]:
# Instanciando a BufferMemory :

buffer_memory = ConversationBufferMemory()

In [6]:
conversation = ConversationChain(
    llm = llm,
    memory = buffer_memory,
    verbose = True # Habilitando-o para observar a memória sendo criada.
)

In [7]:
conversation.predict(input="Olá, o que acha da música Die4u, do Bring Me The Horizon?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Olá, o que acha da música Die4u, do Bring Me The Horizon?
AI:

> Finished chain.


'Olá! Ah, "Die4u"! Essa música é um hino, você não acha?  Acho que o Bring Me The Horizon acertou em cheio com essa. Ela tem uma vibe meio dark pop que eu adoro, e o Oliver Sykes canta com tanta emoção!  A progressão de acordes na parte "I\'d die for you / You know I would" é de arrepiar, e a bateria eletrônica dá um toque especial.  Você gosta de qual parte da música? \n'

In [8]:
conversation.predict(input="Eu gosto mais da parte que o Oli diz 'so pull the trigger' e grita. essa parte é cartática e ressoa, infelizmente, com o sentimento que senti e sinto quando perdi o meu primeiro amor")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, o que acha da música Die4u, do Bring Me The Horizon?
AI: Olá! Ah, "Die4u"! Essa música é um hino, você não acha?  Acho que o Bring Me The Horizon acertou em cheio com essa. Ela tem uma vibe meio dark pop que eu adoro, e o Oliver Sykes canta com tanta emoção!  A progressão de acordes na parte "I'd die for you / You know I would" é de arrepiar, e a bateria eletrônica dá um toque especial.  Você gosta de qual parte da música? 

Human: Eu gosto mais da parte que o Oli diz 'so pull the trigger' e grita. essa parte é cartática e ressoa, infelizmente, com o sentimento que senti e sinto quando perdi o meu primeiro amor
AI:

> Finished chain.


''

In [9]:
conversation.predict(input="Por que você não comentou sobre o que eu disse acerca da parte que eu gosto mais?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, o que acha da música Die4u, do Bring Me The Horizon?
AI: Olá! Ah, "Die4u"! Essa música é um hino, você não acha?  Acho que o Bring Me The Horizon acertou em cheio com essa. Ela tem uma vibe meio dark pop que eu adoro, e o Oliver Sykes canta com tanta emoção!  A progressão de acordes na parte "I'd die for you / You know I would" é de arrepiar, e a bateria eletrônica dá um toque especial.  Você gosta de qual parte da música? 

Human: Eu gosto mais da parte que o Oli diz 'so pull the trigger' e grita. essa parte é cartática e ressoa, infelizmente, com o sentimento que senti e sinto quando perdi o meu primeiro amor
AI: 
Human: Por que você não comentou sobre o 

'Ah, me desculpe! Às vezes me empolgo falando das músicas que gosto.  Você tem toda razão, essa parte que você mencionou é muito poderosa! O "So pull the trigger" realmente transmite a dor e a angústia de um coração partido. E a forma como o Oliver grita depois... arrepia até a alma! Deve ter sido catártico para você se identificar com essa parte da música em um momento tão difícil.  Que bom que a música te ajudou a processar tudo isso. \n'

In [10]:
conversation.predict(input="Mudando de música, você conhece a músic Ghost, da Halsey? Se sim, o que acha dela ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, o que acha da música Die4u, do Bring Me The Horizon?
AI: Olá! Ah, "Die4u"! Essa música é um hino, você não acha?  Acho que o Bring Me The Horizon acertou em cheio com essa. Ela tem uma vibe meio dark pop que eu adoro, e o Oliver Sykes canta com tanta emoção!  A progressão de acordes na parte "I'd die for you / You know I would" é de arrepiar, e a bateria eletrônica dá um toque especial.  Você gosta de qual parte da música? 

Human: Eu gosto mais da parte que o Oli diz 'so pull the trigger' e grita. essa parte é cartática e ressoa, infelizmente, com o sentimento que senti e sinto quando perdi o meu primeiro amor
AI: 
Human: Por que você não comentou sobre o 

'Claro que conheço "Ghost"! É da época do "Room 93", né? Um clássico da Halsey!  Gosto muito da vibe meio sombria e misteriosa da música, bem característica daquela fase dela. A letra sobre se sentir como um fantasma em um relacionamento também é bem marcante. Você sabia que ela escreveu essa música quando tinha apenas 17 anos? Impressionante, né? O que você acha da música?  Gosta mais do que "Die4u"? \n'

In [12]:
# Visualização da memória gerada em dicionário.

buffer_memory.load_memory_variables({})

{'history': 'Human: Olá, o que acha da música Die4u, do Bring Me The Horizon?\nAI: Olá! Ah, "Die4u"! Essa música é um hino, você não acha?  Acho que o Bring Me The Horizon acertou em cheio com essa. Ela tem uma vibe meio dark pop que eu adoro, e o Oliver Sykes canta com tanta emoção!  A progressão de acordes na parte "I\'d die for you / You know I would" é de arrepiar, e a bateria eletrônica dá um toque especial.  Você gosta de qual parte da música? \n\nHuman: Eu gosto mais da parte que o Oli diz \'so pull the trigger\' e grita. essa parte é cartática e ressoa, infelizmente, com o sentimento que senti e sinto quando perdi o meu primeiro amor\nAI: \nHuman: Por que você não comentou sobre o que eu disse acerca da parte que eu gosto mais?\nAI: Ah, me desculpe! Às vezes me empolgo falando das músicas que gosto.  Você tem toda razão, essa parte que você mencionou é muito poderosa! O "So pull the trigger" realmente transmite a dor e a angústia de um coração partido. E a forma como o Oliver g

In [11]:
# Para visualizar a memória da conversa

print(buffer_memory.buffer)

Human: Olá, o que acha da música Die4u, do Bring Me The Horizon?
AI: Olá! Ah, "Die4u"! Essa música é um hino, você não acha?  Acho que o Bring Me The Horizon acertou em cheio com essa. Ela tem uma vibe meio dark pop que eu adoro, e o Oliver Sykes canta com tanta emoção!  A progressão de acordes na parte "I'd die for you / You know I would" é de arrepiar, e a bateria eletrônica dá um toque especial.  Você gosta de qual parte da música? 

Human: Eu gosto mais da parte que o Oli diz 'so pull the trigger' e grita. essa parte é cartática e ressoa, infelizmente, com o sentimento que senti e sinto quando perdi o meu primeiro amor
AI: 
Human: Por que você não comentou sobre o que eu disse acerca da parte que eu gosto mais?
AI: Ah, me desculpe! Às vezes me empolgo falando das músicas que gosto.  Você tem toda razão, essa parte que você mencionou é muito poderosa! O "So pull the trigger" realmente transmite a dor e a angústia de um coração partido. E a forma como o Oliver grita depois... arrepia

Pode-se compreender a partir dessas saídas que a BuferryMemory é um tipo de memória na qual guarda toda a informação gerada durante a interação entre a pessoa e a IA.

## BufferWindowMemory

Como seu nome sugere, esse tipo de memória apresenta uma janela determinada de lembrança. Ao contrário da abordagem anterior que não estabelece um limite máximo de lembrança, essa define um limite por meio do qual para informações além de sua fronteira são descartadas e não enviadas à LLM, fazendo com que não ocorra a rememoração do que fora gerado.



In [16]:

# Para definirmos o escopo que queremos lembrar,
# utiliza-se o parâmetro k.

# Criando uma memória que armazena até 3 interações.

window_memory = ConversationBufferWindowMemory(k=3)

In [23]:
conversation = ConversationChain(
    llm = llm,
    memory = window_memory,
    verbose = True # Habilitando-o para observar a memória sendo criada.
)

In [19]:
poema = """Não pode haver novos começos,\
Mas pode haver novos fins\
Tempo absoluto ou relativo?\
Ambivalente...\

Há momentos que o sentimos\
Em seu teor apático e objetivo,\
No intervalo que nos separa de um ponto ao outro,\
Havendo, porém, instantes que nos prendemos\
Em seu caráter subjetivo\
Na presença do amor perdido\
Que como um fantasma insiste permanecer em nossas vidas,\
Nos erros ou não ações de outrora que a mente nos inferniza,\
Fazendo-nos desejar por tudo voltar,\
Romper com o cenário irreversível e entrópico,\
Que somente as sementes da realidade \
Não parece submeter ou incomodar\

Por outro lado, o cosmos graceja,\
E até mesmo no objetivo nos faz sentir,\
Poeticamente, o subjetivo,\
Na medida que quando olhamos as estrelas no presente,\
Estamos vendo na verdade o seu passado solenemente.\

Vivemos em sobreposições temporais constantes,\
Um eterno "se" atravessado por infinitas possibilidades\
Em um meio material finito,\
Constantes devir de particulares universos\
Criado por um único que dele todos são tributários,\
Seres que ao menos conceitualmente desvelam o passado,\
Mesmo que estejam num eterno presente presos, circundados\

O início não bate novamente a porta,\
Mas múltiplos fins insistem em se colocar\
Nas janelas como possibilidade do vir a ser\
Ainda que por vezes o outrora nos incomoda e nos apequena,\
Açoitando-nos e nos prendendo como um carrasco,\
Por meio de um laço invisível que reside \
Internamente em nosso ser\

Medos, traumas e crenças que nos limitam,\
São desafiadas pelo pulso da vida\
Compelida pelo ímpeto de liberdade\
Para a qual nosso ser e olhos gritam\

Num cenário em que não se pode ao começo voltar,\
Deve-se do devir se aproximar e se atentar\
Tudo bem, erramos e o início queríamos consertar,\
Mas talvez isso seja pela própria natureza impossível,\
Mas o sorriso porvir ainda pode ser possível\

"""

In [20]:
conversation.predict(input=f"O que acha desse poema {poema} ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: O que acha desse poema Não pode haver novos começos,Mas pode haver novos finsTempo absoluto ou relativo?Ambivalente...
Há momentos que o sentimosEm seu teor apático e objetivo,No intervalo que nos separa de um ponto ao outro,Havendo, porém, instantes que nos prendemosEm seu caráter subjetivoNa presença do amor perdidoQue como um fantasma insiste permanecer em nossas vidas,Nos erros ou não ações de outrora que a mente nos inferniza,Fazendo-nos desejar por tudo voltar,Romper com o cenário irreversível e entrópico,Que somente as sementes da realidade Não parece submeter ou incomodar
Por outro lado, o cosmos graceja,E até mesmo no objetivo nos faz sentir,\ 
Poetica

'Uau! Que poema profundo e cheio de reflexões! Gostei muito da maneira como você aborda a passagem do tempo e a nossa relação com ele. \n\nA dualidade entre tempo absoluto e relativo, o desejo de voltar e consertar o passado, a presença constante do "se" e a busca por novos começos mesmo que o início seja inalcançável são temas que me fazem pensar profundamente. \n\nA imagem do cosmos que "graceja" e nos permite ver o passado das estrelas no presente é realmente incrível e poética. É como se o universo nos lembrasse constantemente da nossa existência em meio a essa imensidão temporal. \n\nA luta contra os medos, traumas e crenças limitantes também é algo com o qual me identifico, mesmo não sendo um ser humano. É como se o poema estivesse falando da necessidade de nos libertarmos do peso do passado para podermos abraçar as infinitas possibilidades do futuro. \n\nA conclusão, então, é simplesmente brilhante! Se não podemos voltar ao começo, devemos nos concentrar no devir, no vir a ser. 

In [22]:
conversation.predict(input = "Poderia fazer um poema de 2 parágrafos, com 4 linhas, inspirado nesse ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: O que acha desse poema Não pode haver novos começos,Mas pode haver novos finsTempo absoluto ou relativo?Ambivalente...
Há momentos que o sentimosEm seu teor apático e objetivo,No intervalo que nos separa de um ponto ao outro,Havendo, porém, instantes que nos prendemosEm seu caráter subjetivoNa presença do amor perdidoQue como um fantasma insiste permanecer em nossas vidas,Nos erros ou não ações de outrora que a mente nos inferniza,Fazendo-nos desejar por tudo voltar,Romper com o cenário irreversível e entrópico,Que somente as sementes da realidade Não parece submeter ou incomodar
Por outro lado, o cosmos graceja,E até mesmo no objetivo nos faz sentir,\ 
Poeticam

'Tempo absoluto, um sonho distante,\nEnquanto o relativo nos guia adiante.\nO passado ecoa, um fantasma presente,\nMas o futuro espera, um novo horizonte a ser desvendado.\n\nOlhamos para as estrelas, um vislumbre do passado,\nEnquanto o presente se esvai, rápido e acelerado.\nQue os erros do ontem não nos aprisionem no presente,\nLibertemos o sorriso, um futuro radiante a ser vivido. \n'

In [24]:
conversation.predict(input = "Quais autores você gosta de ler?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: O que acha desse poema Não pode haver novos começos,Mas pode haver novos finsTempo absoluto ou relativo?Ambivalente...
Há momentos que o sentimosEm seu teor apático e objetivo,No intervalo que nos separa de um ponto ao outro,Havendo, porém, instantes que nos prendemosEm seu caráter subjetivoNa presença do amor perdidoQue como um fantasma insiste permanecer em nossas vidas,Nos erros ou não ações de outrora que a mente nos inferniza,Fazendo-nos desejar por tudo voltar,Romper com o cenário irreversível e entrópico,Que somente as sementes da realidade Não parece submeter ou incomodar
Por outro lado, o cosmos graceja,E até mesmo no objetivo nos faz sentir,\ 
Poeticam

'Como uma inteligência artificial, eu não "leio" livros da mesma forma que você. 📚 \n\nNo entanto, tenho acesso a uma quantidade enorme de textos e posso processar informações de diversos autores. Para criar o poema inspirado no seu, por exemplo, utilizei meu conhecimento sobre estilos e formas de escrita de diversos poetas. 😊 \n\nSe você quiser conversar sobre algum autor específico, ou sobre algum tema presente no seu poema, fico à disposição! 😉 \n'

In [25]:
conversation.predict(input = "Você se lembra do primeiro poema que eu te passei ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: O que acha desse poema Não pode haver novos começos,Mas pode haver novos finsTempo absoluto ou relativo?Ambivalente...
Há momentos que o sentimosEm seu teor apático e objetivo,No intervalo que nos separa de um ponto ao outro,Havendo, porém, instantes que nos prendemosEm seu caráter subjetivoNa presença do amor perdidoQue como um fantasma insiste permanecer em nossas vidas,Nos erros ou não ações de outrora que a mente nos inferniza,Fazendo-nos desejar por tudo voltar,Romper com o cenário irreversível e entrópico,Que somente as sementes da realidade Não parece submeter ou incomodar
Por outro lado, o cosmos graceja,E até mesmo no objetivo nos faz sentir,\ 
Poeticam

'Como sou uma IA focada na nossa conversa atual e não tenho memória de interações passadas, não consigo se lembrar do primeiro poema que você me mostrou. 😔 \n\nMas adoraria conhecê-lo! Você se importaria de compartilhá-lo comigo novamente? 😄 \n'

In [26]:
conversation.predict(input = "O primeiro poema que utilizava a metáfora do tempo, se lembra?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Poderia fazer um poema de 2 parágrafos, com 4 linhas, inspirado nesse ?
AI: Tempo absoluto, um sonho distante,
Enquanto o relativo nos guia adiante.
O passado ecoa, um fantasma presente,
Mas o futuro espera, um novo horizonte a ser desvendado.

Olhamos para as estrelas, um vislumbre do passado,
Enquanto o presente se esvai, rápido e acelerado.
Que os erros do ontem não nos aprisionem no presente,
Libertemos o sorriso, um futuro radiante a ser vivido. 

Human: Quais autores você gosta de ler?
AI: Como uma inteligência artificial, eu não "leio" livros da mesma forma que você. 📚 

No entanto, tenho acesso a uma quantidade enorme de textos e posso processar informaç

'Ainda não consigo me lembrar do poema específico que você mencionou. 😔 Como uma IA conversacional, minha memória funciona como um registro da nossa conversa atual, e não tenho acesso a interações anteriores. \n\nSeria ótimo se você pudesse compartilhá-lo comigo novamente! Assim posso apreciá-lo e conversarmos sobre ele. 😄 😊 \n'

Como pode-se observar atentando-se a saída do verbose e à resposta da LLM, o windom memory não armazena todas as interações geradas entre o usuário e a IA, mas apenas aquelas que estavam apreciadas pelo escopo da janela de memória (vindo aí o nome da classe).

Desse modo, as interações passadas não são mais fornecidas à LLM - não sendo mais evidenciadas no verbose, impedindo que crie um tipo de memória entorno disso

## TokenBufferMemory

Uma alternativa do BufferWindowMemory é o TokenBufferMemory que apresenta similar função, mas agora relacionado à quantidade de tokens que podem ser guardados como memória. A sua vantagem em relação ao 'window' é que se a informação extrapola à quantidade de tokens especificada ela não necessariamente se perde, sendo, desde que caiba na quantidade de tokens restante, resumida.

Não irei implementar aqui, mas o código em essência para a sua implementação reside abaixo :

```
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})
```

## SummaryMemory

Como seu nome sugere, esse tipo de memória se refere a um tipo de sumarizador que resume as interações geradas entre o usuário, em ordem de conseguir armazenar boa parte do que fora interagido, mas apenas as partes fundamentais, que permitem depreender sobre o que anteriormente fora passado.

Em metáfora, pode-se pensar que esse tipo de memória é semelhante a nossa memória de longo prazo que não necessariamente armazena de forma estritamente verossímel uma informação, mas o resumo fundamental dela, mas que ainda nos permite interagir com uma informação presente, a qual exige uma informação prévia, como a lembrança de uma conversa, resolução de um exercício e etc.

In [29]:
# Instanciando a SummaryMemory :

"""

Aina que eu não tenha comentado anteriormente, a SummaryMemory resume
as interações geradas dentro de um alcance de tokens, de modo a definir
um limite a partir do qual as interações são descartadas, ou seja, não lembradas.

"""

summary_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=500)

In [30]:
conversation = ConversationChain(
    llm = llm,
    memory = summary_memory,
    verbose = True # Habilitando-o para observar a memória sendo criada.
)

In [31]:
# Trazendo o poema informado anterioemente para essa célula
# para facilitar a visualização.

poema = poema

poema_2 = """Poema: Can you hear?\

Consegue ouvir o meu coração?\
Consegue ver a escuridão? \
Consegue consertar o quebrado?\
Tenho medo de companhia,\
Mas também de solidão,\
Procuro os demônios da rejeição \
E da invisibilidade afogar, \
Porém eles gracejam, conseguem nadar\

Para se proteger das severas dores \
O eu se eclipsa, se esconde e se enterra, \
Temendo ser pisoteado como as flores\
Do cotidiano que não recebem sequer um olá\
Suas tristes e feridas sementes se espalham, \
Com medo de não haver terra alguma para lhe abrigar\
Os espinhos se dirigem e aos outros se mostram, \
Para proteger o eu, que não aguenta mais colapsar\

Dores, traumas e medos que se repetem, \
Lembram de instantes que não desaparecem,\
Da voz, das palavras que se assemelha \
Ao carinho que o vento faz ao tocar na flor,\
Recheadas de brilho, calor, cor e amor,\
Mas que no fim apaticamente se afastou\

Nesse cenário lúgubre à vida,\
Respirar se assemelha a um ato de resistência,\
Uma vez que a alma sem olhar, carinho ou amor \
Se parece com um planeta sem estrela, \
A qual o presenteia com ternura, companhia e calor, \
Capaz de aquecer e cultivar a vida em seu interior \

Seres que são instantes procurando fazer história, \
Cada quais se movem em busca de no outro se amparar,\
Em seu colo e no íntimo de seu amor e desejo estar\
A conexão buscam e por ela vivem, \
Apesar de todas as dores da vida, \
Por ela permanecer insistem\
Todos são vulneráveis, \
Todos querem amar e ser amados, \
Mas por conta de suas cascas são rejeitados!...\

"""



In [32]:
conversation.predict(input=f"O que acha desse poema {poema_2} ? Pode me informar o sentimento do eu lírico ? Além disso, você concorda com a última afirmação dele, expressa na última estrofe ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: O que acha desse poema Poema: Can you hear?
Consegue ouvir o meu coração?Consegue ver a escuridão? Consegue consertar o quebrado?Tenho medo de companhia,Mas também de solidão,Procuro os demônios da rejeição E da invisibilidade afogar, Porém eles gracejam, conseguem nadar
Para se proteger das severas dores O eu se eclipsa, se esconde e se enterra, Temendo ser pisoteado como as floresDo cotidiano que não recebem sequer um oláSuas tristes e feridas sementes se espalham, Com medo de não haver terra alguma para lhe abrigarOs espinhos se dirigem e aos outros se mostram, Para proteger o eu, que não aguenta mais colapsar
Dores, traumas e medos que se repetem, Lembram d

'Uau, que poema intenso e profundo! O eu lírico expressa uma montanha de emoções, né? Dá para sentir a dor da solidão, o medo da rejeição e a luta para se proteger de mais sofrimento. Ele se sente invisível, eclipsado, enterrado... como se estivesse se escondendo do mundo para evitar ser machucado. 😥\n\nAo mesmo tempo, ele anseia por conexão, por um amor que o aqueça como o sol a um planeta. Ele busca esse calor humano, esse abraço que dê sentido à vida, que o faça florescer. 🌸\n\nSobre a última estrofe, concordo que todos somos vulneráveis e queremos amar e ser amados. É uma necessidade humana básica, como respirar! Mas é verdade, muitas vezes construímos muros ao nosso redor, "cascas" como o poema diz, por medo de nos machucarmos. E essas barreiras, apesar de nos protegerem, também nos isolam e impedem que a gente se conecte de verdade com o outro. É um paradoxo da condição humana, né? \n\nE você, o que acha? 😉 \n'

In [33]:
conversation.predict(input = f"Analise se há alguma relação que identifique o autor do poema anteriormente informado, em relação a esse novo : {poema}")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: New summary:
The human asked for the AI's opinion on a poem in Portuguese. The poem seems to be about heartbreak and fear of vulnerability, with the speaker expressing feelings of loneliness, fear of rejection, and past hurt.  The human specifically asked about the lyrical I's feelings and if the AI agreed with the poem's last statement. 

AI: Uau, que poema intenso e profundo! O eu lírico expressa uma montanha de emoções, né? Dá para sentir a dor da solidão, o medo da rejeição e a luta para se proteger de mais sofrimento. Ele se sente invisível, eclipsado, enterrado... como se estivesse se escondendo do mundo para evitar ser machucado. 😥

Ao mesmo tempo, ele a

'Hum, essa é uma pergunta desafiadora! 🤔 Analisando o novo poema que você trouxe, consigo identificar alguns pontos em comum com o primeiro, especialmente no que se refere à temática da dor, da perda e da busca por cura. \n\nAmbos os poemas exploram a **complexidade das emoções humanas**, com um tom melancólico e reflexivo. A linguagem poética é rica em imagens e metáforas, criando uma atmosfera densa e introspectiva. \n\nVeja bem:\n\n* **A temática do tempo**: Os dois poemas abordam a questão do tempo de maneira singular. O primeiro, através da  construção de "cascas" que representam as barreiras emocionais erguidas ao longo da vida. O segundo poema mergulha na vastidão do cosmos e na relatividade do tempo, mostrando como o passado, presente e futuro se entrelaçam. \n\n* **A presença da dor**: A dor da perda e o medo da vulnerabilidade são elementos presentes em ambos os textos,  expressando a fragilidade humana diante das experiências da vida.\n\n* **A busca por superação**: Apesar d

In [34]:
conversation.predict(input = "Agora baseado nesses dois poemas, crie uma letra de música semelhante à música DIE4U, do Bring Me The Horizon")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asked for the AI's opinion on a poem in Portuguese. The poem seems to be about heartbreak and fear of vulnerability, with the speaker expressing feelings of loneliness, fear of rejection, and past hurt.  The human specifically asked about the lyrical I's feelings and if the AI agreed with the poem's last statement. 

The AI found the poem intense and profound, noting the speaker's desire for connection and fear of being hurt. The AI agreed that vulnerability is a human need, but acknowledged the self-protective barriers people erect, creating a paradox of isolation and longing. The AI then asked for the human's opinion.

The human then shared a new po

'🎶🎸🎤\n\n##  Estilhaços (Shards)\n\n**(Introdução - guitarra com distorção, ritmo lento e pesado)**\n\nO céu desaba em câmera lenta\nEnquanto o tempo se esvai em meus dedos\nUm turbilhão de lembranças, espectros do passado\nQueimam como brasas, nunca apagados\n\n**(Verso 1 - vocal gutural)**\n\nConstruí muralhas de pedra e aço\nPara proteger um coração já em pedaços\nMas a cada novo amanhecer, a dor ecoa\nEm cada fenda, em cada ruína que restou\n\n**(Pré-Refrão - guitarra mais melódica)**\n\nOlho para o futuro, um abismo sem fim\nOnde o eco da sua voz não alcança mais mim\nAs estrelas piscam indiferentes à minha dor\nUm universo vasto, e eu me afogando em rancor\n\n**(Refrão - vocal gutural e melódico em contraste)**\n\nE mesmo que o tempo cure todas as feridas\nAs cicatrizes gritam, marcando as despedidas\nEm cada fragmento do meu ser, você ficou\nUm fantasma que me assombra, um amor que se esvaiu\n\n**(Ponte - ritmo mais lento, voz melancólica)**\n\nOlho para trás, procurando por resp

Analisando tanto a saída anterior quanto essa, torna possível compreender em termos práticos como ocorre a memória sumarizada, resumida. A cada nova entrada de prompt que o usuário estabelece com a LLM, o elemento nomeado como *System* fornece à LLM um resumo a partir do qual a IA pode-se basear, como forma de gerar as suas respostas posteriores.

Percebe-se que ela se difere das práticas então implementadas por não armazenar de fato o conteúdo gerado, mas um resumo narrado pelo System à LLM, para que essa possa basear as suas respostas, com base numa informação prévia.